In [21]:
import pandas as pd
import csv
import json


def get_handle(url):
    splits = url.split('/')
    if url:
        if url.endswith('/'):
            handle = splits[-2]
        else:
            handle = splits[-1]
    else:
        handle = None    
    return handle
        
df = pd.read_csv('influencer_list.csv')
for index, row in df.iterrows():
    influencer = row['Influencer']
    category = row['Category']
    
    twitter_url = row['Twitter']
    twitter_handle = get_handle(twitter_url)
    
    instagram_url = row['Instagram']
    
    if instagram_url != 'null':
        instagram_handle = get_handle(instagram_url)
        json_location = 'instagram_data/{}/{}/{}.json'.format(category.lower(), instagram_handle, instagram_handle)

        f = open(json_location, 'r')
        items = json.load(f)
        f.close()
        
        outfile_location = 'instagram-data/{}'.format(twitter_handle)
        with open(outfile_location, 'w') as insta_file:
            for item in items:
                post = dict()

                post['twitter_handle'] = twitter_handle
                post['instagram_handle'] = instagram_handle

                try:
                    post['likes'] = item['edge_media_preview_like']['count']
                except (KeyError, IndexError) as err:
                    post['likes'] = None

                try:
                    post['comments'] = item['edge_media_to_comment']['count']
                except (KeyError, IndexError) as err:
                    post['comments'] = None

                try:
                    post['hashtags'] = item['tags']
                except (KeyError, IndexError) as err:
                    post['hashtags'] = None

                try:
                    post['caption'] = item['edge_media_to_caption']['edges'][0]['node']['text']
                except (KeyError, IndexError) as err:
                    post['caption'] = None

                try:
                    post['timestamp'] = item['taken_at_timestamp']
                except (KeyError, IndexError) as err:
                    post['timestamp'] = None

                try:
                    post['image_thumbnail'] = item['thumbnail_src']
                except (KeyError, IndexError) as err:
                    post['image_thumbnail'] = None
                
                insta_file.write(json.dumps(post))
                insta_file.write('\n')

In [ ]:
from bs4 import BeautifulSoup
import requests

page_content = requests.get("https://www.instagram.com/aringhosh/").content
soup = BeautifulSoup(page_content, 'html.parser')

# for ultag in soup.find_all('ul', {'class': '_h9luf'}):
#     for litag in ultag.find_all('li'):
#         print(litag.text)

# soup.find_all('ul', {'class': '_h9luf'})
print(page_content.)

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import csv
import json


def get_handle(url):
    splits = url.split('/')
    if url:
        if url.endswith('/'):
            handle = splits[-2]
        else:
            handle = splits[-1]
    else:
        handle = None    
    return handle
        
columns = ['tw_handle','ig_handle','ig_followers', 'ig_following', 'ig_posts', 'ig_thumbnail_url']
outdf = pd.DataFrame(columns=columns)

    
df = pd.read_csv('influencer_list.csv')
for index, row in df.iterrows():
    influencer = row['Influencer']
    category = row['Category']
    
    twitter_url = row['Twitter']
    twitter_handle = get_handle(twitter_url)
    
    instagram_url = row['Instagram']

    if instagram_url != 'null':
        instagram_handle = get_handle(instagram_url)
        
        response = requests.get(instagram_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        metas = soup.find_all('meta')
        description = [ meta.attrs['content'] for meta in metas if 'name' in meta.attrs and meta.attrs['name'] == 'description' ][0]
        numbers = re.findall(r'\d+', description)

        followers = numbers[0]
        following = numbers[1]
        posts = numbers[2]

        thumbnail_url = soup.find("meta",  property="og:image")['content']

        result_list = [twitter_handle, instagram_handle, followers, following, posts, thumbnail_url]
        outdf.loc[index] = result_list

# outdf.to_csv('instagram_influencers_details.csv', sep=';', index=False)
print(outdf)

          tw_handle             ig_handle ig_followers ig_following ig_posts  \
0        fitmencook            FitMenCook            1            2      511   
1     yolanda_gampp         yolanda_gampp            2            8      841   
2        izyhossack            izyhossack          221            9      795   
3   NatashasKitchen       natashaskitchen          124            2      231   
4      inspiralized          inspiralized          188            8      373   
5    rosannapansino        rosannapansino            3            5      117   
6    smittenkitchen        smittenkitchen          735            5        1   
7         sweetambs             sweetambs            1            1      921   
8   CupcakeAddictAU    mycupcakeaddiction            1            2      486   
9       joythebaker           joythebaker          427            4      657   
10            MKBHD                 MKBHD          298          710        3   
11         ijustine              ijustin

In [ ]:
outdf.shape

In [ ]:
# Image downloader
import logging
import requests
import pandas as pd
from os import listdir
from os.path import isfile, join

LOG_FILENAME = 'instagram_downloader_log.out'
logging.basicConfig(
    filename=LOG_FILENAME,
    level=logging.ERROR,
)

path = 'instagram-data'
instagram_files = [f for f in listdir(path) if isfile(join(path, f)) and (not f.startswith('.'))]

counter = 0
for instagram_file in instagram_files:
    file_name = join(path, instagram_file)
    df = pd.read_json(file_name, lines=True)
    directory = 'instagram-images/{}'.format(instagram_file)
    if not os.path.exists(directory):
            os.makedirs(directory)
    for index, row in df.iterrows():
        image_url = row['image_thumbnail']
        response = requests.get(image_url, stream=True)
        if not response.ok:
            logging.error('Response {};Influencer:{};URL:{}'.format(
            response, instagram_file, image_url))
        else:
            with open('instagram-images/{}/{}_{}.jpg'.format(instagram_file, instagram_file, index), 'wb') as handle:
                for block in response.iter_content(1024):
                    if not block:
                        break
                    handle.write(block)
#             logging.debug('SUCCESS;Influencer:{};URL:{};index:{}'.format(
#             response, instagram_file, image_url, index))
    counter += 1
    print("Done for: {}".format(instagram_file))